In [176]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [177]:
#!python
from numpy import *
from numpy.random import *
%matplotlib inline
import os
import time
import numpy as np
import pandas as pd
import sys
import cPickle as pickle
from   random import *
import scipy.stats
import matplotlib.pyplot as plt
sys.path.insert(1,'./code')
from visualize import plot_raw_data, get_population_data, plot_results
from EM import TransformToProb, RunEM
import pyparticleest.models.nlg as nlg
import pyparticleest.simulator as simulator
import pyparticleest.utils.kalman as kalman
from scipy.stats import bernoulli
import pyparticleest.interfaces as interfaces
from scipy.optimize import newton

In [178]:
df = pd.read_csv('thaldata.csv',header=None)
Y = df.values.reshape(3000)
T = len(Y)

In [179]:
# Forward particles
N = 100
# Backward trajectories
M = 10
# 
P0 = 0.1
Q = 0.1
#trial
trial = 50
#
coefficient = scipy.special.binom(trial,Y)
#
l2pi = math.log(2 * math.pi)

In [180]:
def resample(weights):
  n = len(weights)
  indices = []
  C = [0.] + [sum(weights[:i+1]) for i in range(n)]
  u0, j = random(), 0
  for u in [(u0+i)/n for i in range(n)]:
    while u > C[j]:
      j+=1
    indices.append(j-1)
  return indices

In [181]:
def measure(X, Y, trial, t, coefficient):
    # log-pdf of p(y_t|x_t)
    return np.log(coefficient[t]) + X[t,:] * Y[t] - trial * np.log(1+np.exp(X[t,:]))

In [144]:
def forward(X, a, S, t):
    #log-pdf of p(x_t|x_{t-1})
    return -0.5 * (l2pi + np.log(S) + ((X[t,:]-a).ravel() ** 2) / S)

In [145]:
def lognormpdf_scalar(err, S):
    """
    Calculate gaussian probability density of all elements in err, when
    err[i] ~ N(0,S) and each element in err is a scalar
    """
    return -0.5 * (l2pi + np.log(S) + (err.ravel() ** 2) / S)

In [372]:
##Initialization
t = 0
v = np.zeros(N)
X = np.empty([T,N])
ancestor = np.empty([T,N])
w = np.empty([T,N])
W = np.empty([T,N])
X[t,:] = np.random.normal(0, np.sqrt(P0), N)
w[t,:] = measure(X, Y, trial, t, coefficient)
W[t,:] = np.exp(w[t,:] - np.max(w[t,:]))
W[t,:] /= np.sum(W[t,:])
ancestor[t,:]= X[t,resample(W[t,:])]

In [307]:
var(ancestor[0,:])

0.0057450670748890945

In [341]:
mode = np.empty(T)
variance = np.empty(T)

In [373]:
%%time
for tt in range(T-1):
    t=tt+1
    samplemean = nanmean(ancestor[tt,:])
    samplevariance = nanvar(ancestor[tt,:]) + Q
    mode[tt] = samplemean + (samplevariance)*(Y[t] - np.exp(samplemean)/(1 + np.exp(samplemean)) * trial)
    variance[tt] = 1.0 / (1.0/((samplevariance) + trial * np.exp(samplemean)/(1+np.exp(samplemean))**2))
    X[t,:] = np.random.normal(mode[tt],np.sqrt(variance[tt]), N)
    #a = ancestor[tt,:]
    #X[t,:] = a + np.random.normal(0, np.sqrt(Q), N) ##f(x_t|x_{t-1}) standard deviation
    w[t,:] = measure(X, Y, trial, t, coefficient) + forward(X, ancestor[tt,:], Q, t) - lognormpdf_scalar(X[t,:]-mode[tt], variance[tt])   
    #w[t,:] = measure(X, Y, trial, t, coefficient) ##g(y_t|x_t)
    W[t,:] = np.exp(w[t,:] - np.max(w[t,:]))
    W[t,:] /= np.sum(W[t,:])
    ancestor[t,:]= X[t,resample(W[t,:])]

CPU times: user 4.31 s, sys: 43.1 ms, total: 4.35 s
Wall time: 4.41 s


In [374]:
maxlogweight = np.max(w,axis=1)
weight = np.exp(w - np.reshape(np.repeat(maxlogweight, N),(T, N)))
np.sum(np.log(np.sum(weight, axis = 1)/N))+np.sum(maxlogweight)

-3343.1201180082153

0.90707138892330608

In [377]:
variance[T-1]

-1.4129910999481303

In [ ]:
for tt in range(2999):
    t=tt+1
    a = ancestor[tt,:]
    mode = a + (Q + v)*(Y[t] - np.exp(a)/(1 + np.exp(a)) * trial)
    print mode
    variance = 1.0 / (1.0/((Q + v) + trial * np.exp(a)/(1+np.exp(a))**2))
    v = variance
    X[t,:] = np.random.normal(mode,np.sqrt(variance), N)
    #X[t,:] = a + np.random.normal(0, np.sqrt(Q), N) ##f(x_t|x_{t-1}) standard deviation
    w[t,:] = measure(X, Y, trial, t, coefficient) + forward(X, a, Q, t) - lognormpdf_scalar(X[t,:]-mode, variance)   
    #w[t,:] = measure(X, Y, trial, t, coefficient) ##g(y_t|x_t)
    W[t,:] = np.exp(w[t,:] - np.max(w[t,:]))
    W[t,:] /= np.sum(W[t,:])
    ancestor[t,:]= X[t,resample(W[t,:])]

In [ ]:
print measure(X, Y, trial, 1000, coefficient)
print coefficient[1000]
print forward(X, ancestor[999,:], Q, 1000)
print lognormpdf_scalar(X[1000,:]-mode, variance)

In [ ]:
## for t > 0
for tt in range(1):
    t=tt+1
    a = ancestor[t,:]
    mode = a + t*Q*(Y[t] - np.exp(a)/(1 + np.exp(a)) * (Y[t] - trial * (1+np.exp(a))))
    variance = 1.0 / (t*Q + Y[t]*np.exp(a)/(1+np.exp(a))**2 + trial * np.exp(a)*(1-np.exp(a))/(1+np.exp(a))**3)
    X[t,:] = np.random.normal(mode,variance)
    w[t,:] = measure(X, Y, trial, t, coefficient) + forward(X, a, Q, t) - lognormpdf_scalar(X[t,:]-mode, variance)
    W[t,:] = np.exp(w[t,:] - np.max(w[t,:]))
    W[t,:] /= np.sum(W[t,:])
    ancestor[t+1,:]= X[t,resample(W[t,:])]

In [ ]:
1.0 / (t*Q + Y[t]*np.exp(a)/(1+np.exp(a))**2 + trial * np.exp(a)*(1-np.exp(a))/(1+np.exp(a))**3)

In [ ]:
np.exp(a)*(1-np.exp(a))/(1+np.exp(a))**3

In [ ]:
ancestor[1,:]

In [ ]:
def h(x):
    return x - ancestor + t*Q *(y[t] - np.exp(x)/(1+np.exp(x)))/(1+np.exp(x))

def dh(x):
    return 1 - t * Q

In [ ]:
def k(x):
    return x**3 -1

In [ ]:
def dk(x):
    return 3*x**2

In [ ]:
x0=0.4
print newton(k, x0, dk)

In [ ]:
X = np.empty([T,N])
ancestor = np.empty([T,N])
w = np.empty([T,N])
W = np.empty([T,N])
X[t,:] = np.random.normal(0, np.sqrt(P0), N)
w[t,:] = measure(X, Y, trial, t, coefficient)
W[t,:] = np.exp(w[t,:] - np.max(w[t,:]))
W[t,:] /= np.sum(W[t,:])
ancestor[t,:]= X[t,resample(W[t,:])]